<a href="https://colab.research.google.com/github/choiboy92/ImperialML/blob/main/Coursework/ML_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

file1 = 'https://raw.githubusercontent.com/choiboy92/ImperialML/main/Coursework/dataset1.csv'
df = pandas.read_csv(file1)
arm_length = np.array(df['Arm length (m)'])
weight = np.array(df['Ball weight (kg)'])
radius = np.array(df['Ball radius (mm)'])
temp = np.array(df['Air temperature (deg C)'])
k = np.array(df['Spring constant (N per m)'])
dev_weight = np.array(df['Device weight (kg)'])
y = np.array(df['Target hit'])

In [64]:
scArray1 = np.array([[np.mean(arm_length), np.mean(weight), np.mean(radius), np.mean(temp), np.mean(k), np.mean(dev_weight)],
                   [np.std(arm_length), np.std(weight), np.std(radius), np.std(temp), np.std(k), np.std(dev_weight)]])

np.savetxt('scaleParams1.txt', scArray1)

In [65]:
# download it from the Colab interface
from google.colab import files
files.download('scaleParams1.txt')

loadedScales1 = np.loadtxt('scaleParams1.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [95]:
# Normalise input datasets

X_unscaled = np.concatenate((arm_length.reshape(2000,1), weight.reshape(2000,1), radius.reshape(2000,1), temp.reshape(2000,1), k.reshape(2000,1), dev_weight.reshape(2000,1)), axis=1)
X_scaled = (X_unscaled-loadedScales1[0,:])/loadedScales1[1,:]

from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled,y, test_size=0.33)

In [96]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

#set up a sequential neural network
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=6)) 
#model.add(Dense(units=5, activation='relu')) 
model.add(Dense(units=2, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam')

from tensorflow.keras.utils import to_categorical
yt_binary = to_categorical(y_train)
model.fit(X_train, yt_binary, epochs=750, batch_size=64)


Epoch 1/750
21/21 [==============================] - 0s 1ms/step - loss: 0.8242
Epoch 2/750
21/21 [==============================] - 0s 1ms/step - loss: 0.7913
Epoch 3/750
21/21 [==============================] - 0s 1ms/step - loss: 0.7635
Epoch 4/750
21/21 [==============================] - 0s 2ms/step - loss: 0.7397
Epoch 5/750
21/21 [==============================] - 0s 2ms/step - loss: 0.7193
Epoch 6/750
21/21 [==============================] - 0s 2ms/step - loss: 0.7009
Epoch 7/750
21/21 [==============================] - 0s 2ms/step - loss: 0.6833
Epoch 8/750
21/21 [==============================] - 0s 2ms/step - loss: 0.6666
Epoch 9/750
21/21 [==============================] - 0s 2ms/step - loss: 0.6504
Epoch 10/750
21/21 [==============================] - 0s 2ms/step - loss: 0.6344
Epoch 11/750
21/21 [==============================] - 0s 2ms/step - loss: 0.6178
Epoch 12/750
21/21 [==============================] - 0s 2ms/step - loss: 0.6008
Epoch 13/750
21/21 [=================

In [97]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)

nTot = len(y_pred) 
nMatch = 0 
for i in range(len(y_pred)):
  if y_test[i] == y_pred[i]:
    nMatch += 1
print("With Validation data -- Accuracy: ", 100 * nMatch / nTot, "%")

With Validation data -- Accuracy:  92.27272727272727 %


## KFold validation - dataset1

In [98]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
k = 5
kf = KFold(n_splits = k, shuffle = True)
kf_results = np.array([])
for train_index, test_index in kf.split(X_scaled):
  X_train, X_test = X_scaled[train_index], X_scaled[test_index]
  y_train, y_test = y[train_index], y[test_index]
  
  #set up a sequential neural network
  model = Sequential()
  model.add(Dense(units=6, activation='relu', input_dim=6)) 
  #model.add(Dense(units=5, activation='relu')) 
  model.add(Dense(units=2, activation='sigmoid')) 
  model.compile(loss='binary_crossentropy', optimizer='adam')

  yt_binary = to_categorical(y_train)
  model.fit(X_train, yt_binary, epochs=750, batch_size=64)
  y_pred = model.predict(X_test)
  y_pred = np.argmax(y_pred, axis=-1)

  nTot = len(y_pred) 
  nMatch = 0 
  for i in range(len(y_pred)):
    if y_test[i] == y_pred[i]:
      nMatch += 1
  print("With Validation data -- Accuracy: ", 100 * nMatch / nTot, "%")
  kf_results = np.append(kf_results, 100 * nMatch/nTot)

print(k,"-Fold results:", kf_results)
print(k,"-Fold avg:", np.average(kf_results))


Streaming output truncated to the last 5000 lines.
Epoch 504/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1569
Epoch 505/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1567
Epoch 506/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1568
Epoch 507/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1566
Epoch 508/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1567
Epoch 509/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1568
Epoch 510/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1567
Epoch 511/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1566
Epoch 512/750
25/25 [==============================] - 0s 2ms/step - loss: 0.1569
Epoch 513/750
25/25 [==============================] - 0s 3ms/step - loss: 0.1567
Epoch 514/750
25/25 [==============================] - 0s 3ms/step - loss: 0.1566
Epoch 515/750
25/25 [==========================

## Iterations for 1 hidden layer

In [51]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
yt_binary = to_categorical(y_train)

nodes = np.array([2,4,6,8,10,12,14])
epoch_num = np.array([100,250,500,750,1000])
batch_num = np.array([16,32,64])

maxacc = 0
maxacc_params = np.array([])
result = np.array([])
for b in range(0,len(batch_num)):
  e_array = np.array([])
  for e in range(0,len(epoch_num)):
    n_array = np.array([])
    for n in range(0,len(nodes)):
      #set up a sequential neural network
      model = Sequential()
      model.add(Dense(units=nodes[n], activation='relu', input_dim=6)) 
      #model.add(Dense(units=5, activation='relu')) 
      model.add(Dense(units=2, activation='softmax')) 
      model.compile(loss='categorical_crossentropy', optimizer='adam')
      model.fit(X_train, yt_binary, epochs=epoch_num[e], batch_size=batch_num[b])

      y_pred = model.predict(X_test)
      y_pred = np.argmax(y_pred, axis=-1)

      nTot = len(y_pred) 
      nMatch = 0 
      for i in range(len(y_pred)):
        if y_test[i] == y_pred[i]:
          nMatch += 1
      #print("With Validation data -- Accuracy: ", nMatch / nTot, "%")
      if (nMatch / nTot)>maxacc:
        maxacc = (nMatch / nTot)
        maxacc_params = np.append(maxacc_params, [nodes[n], epoch_num[e], batch_num[b]])
      n_array= np.append(n_array, 100 * nMatch / nTot)
    e_array= np.append(e_array, n_array)
  result = np.append(result,e_array)

result.reshape((len(batch_num), len(epoch_num), len(nodes)))
print(np.amax(result))

Streaming output truncated to the last 5000 lines.
21/21 [==============================] - 0s 2ms/step - loss: 0.1435
Epoch 502/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1434
Epoch 503/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1435
Epoch 504/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1439
Epoch 505/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1433
Epoch 506/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1437
Epoch 507/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1437
Epoch 508/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1434
Epoch 509/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1432
Epoch 510/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1434
Epoch 511/1000
21/21 [==============================] - 0s 2ms/step - loss: 0.1432
Epoch 512/1000
21/21 [=============================

In [57]:
print(maxacc_params.reshape(6,3))
#[[  2. 100.  16.]
# [  6. 100.  16.]
# [  6. 250.  16.]
# [ 12. 250.  16.]
# [ 14. 100.  32.]
# [  6. 750.  64.]] --> 93.03%

[[  2. 100.  16.]
 [  6. 100.  16.]
 [  6. 250.  16.]
 [ 12. 250.  16.]
 [ 14. 100.  32.]
 [  6. 750.  64.]]


## For dataset 2

In [99]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

file2 = 'https://raw.githubusercontent.com/choiboy92/ImperialML/main/Coursework/dataset2.csv'
df2 = pandas.read_csv(file2)
arm_length2 = np.array(df2['Arm length (m)'])
weight2 = np.array(df2['Ball weight (kg)'])
radius2 = np.array(df2['Ball radius (mm)'])
temp2 = np.array(df2['Air temperature (deg C)'])
k2 = np.array(df2['Spring constant (N per m)'])
dev_weight2 = np.array(df2['Device weight (kg)'])
y2 = np.array(df2['Target hit'])

In [100]:
scArray2 = np.array([[np.mean(arm_length2), np.mean(weight2), np.mean(radius2), np.mean(temp2), np.mean(k2), np.mean(dev_weight2)],
                   [np.std(arm_length2), np.std(weight2), np.std(radius2), np.std(temp2), np.std(k2), np.std(dev_weight2)]])

np.savetxt('scaleParams2.txt', scArray2)

In [101]:
# download it from the Colab interface
from google.colab import files
files.download('scaleParams2.txt')

loadedScales2 = np.loadtxt('scaleParams2.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [102]:
# Normalise input datasets

X_unscaled2 = np.concatenate((arm_length2.reshape(4000,1), weight2.reshape(4000,1), radius2.reshape(4000,1), temp2.reshape(4000,1), k2.reshape(4000,1), dev_weight2.reshape(4000,1)), axis=1)
X_scaled2 = (X_unscaled2-loadedScales2[0,:])/loadedScales2[1,:]

In [ ]:
from sklearn import model_selection
X_train2, X_test2, y_train2, y_test2 = model_selection.train_test_split(X_scaled2,y2, test_size=0.33)

In [118]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

#set up a sequential neural network
model = Sequential()
model.add(Dense(units=10, activation='tanh', input_dim=6)) 
model.add(Dense(units=20, activation='tanh')) 
model.add(Dense(units=15, activation='tanh')) 
#model.add(Dense(units=124, activation='tanh')) 
model.add(Dense(units=2, activation='sigmoid')) 
model.compile(loss='binary_crossentropy', optimizer='adam')

from tensorflow.keras.utils import to_categorical
yt_binary2 = to_categorical(y_train2)
model.fit(X_train2, yt_binary2, epochs=500, batch_size=32)

y_pred2 = model.predict(X_test2)
y_pred2 = np.argmax(y_pred2, axis=-1)

nTot = len(y_pred2) 
nMatch = 0 
for i in range(len(y_pred2)):
  if y_test2[i] == y_pred2[i]:
    nMatch += 1
print("With Validation data -- Accuracy: ", 100 * nMatch / nTot, "%")


Epoch 1/500
84/84 [==============================] - 1s 2ms/step - loss: 0.7017
Epoch 2/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6875
Epoch 3/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6797
Epoch 4/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6743
Epoch 5/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6698
Epoch 6/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6676
Epoch 7/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6650
Epoch 8/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6631
Epoch 9/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6610
Epoch 10/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6592
Epoch 11/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6580
Epoch 12/500
84/84 [==============================] - 0s 2ms/step - loss: 0.6565
Epoch 13/500
84/84 [=================

In [86]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
k = 5
kf = KFold(n_splits = k, shuffle = True)
kf_results2 = np.array([])
for train_index, test_index in kf.split(X_scaled2):
  X_train2, X_test2 = X_scaled2[train_index], X_scaled2[test_index]
  y_train2, y_test2 = y2[train_index], y2[test_index]
  
  #set up a sequential neural network
  model = Sequential()
  model.add(Dense(units=6, activation='tanh', input_dim=6)) 
  model.add(Dense(units=25, activation='tanh')) 
  #model.add(Dense(units=50, activation='tanh')) 
  #model.add(Dense(units=124, activation='tanh')) 
  model.add(Dense(units=2, activation='sigmoid')) 
  model.compile(loss='binary_crossentropy', optimizer='adam')

  yt_binary2 = to_categorical(y_train2)
  model.fit(X_train2, yt_binary2, epochs=250, batch_size=32)

  y_pred2 = model.predict(X_test2)
  y_pred2 = np.argmax(y_pred2, axis=-1)

  nTot = len(y_pred2) 
  nMatch = 0 
  for i in range(len(y_pred2)):
    if y_test2[i] == y_pred2[i]:
      nMatch += 1
  print("With Validation data -- Accuracy: ", 100 * nMatch / nTot, "%")

  kf_results2 = np.append(kf_results2, 100 * nMatch/nTot)

print(k,"-Fold results:", kf_results2)
print(k,"-Fold avg:", np.average(kf_results2))


Epoch 1/250
100/100 [==============================] - 1s 2ms/step - loss: 0.6937
Epoch 2/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6846
Epoch 3/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6822
Epoch 4/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6806
Epoch 5/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6792
Epoch 6/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6784
Epoch 7/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6773
Epoch 8/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6765
Epoch 9/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6759
Epoch 10/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6754
Epoch 11/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6742
Epoch 12/250
100/100 [==============================] - 0s 2ms/step - loss: 0.6742
Epoch 13/250


In [113]:
# Iterative code
# Main variables are activation function, no. of layers, no. of nodes in each layer

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
yt_binary2 = to_categorical(y_train2)


act_func = ["tanh"]
# for two layers
# each row is combination of nodes to try out
nodes = np.array([[30, 5, 10],
                  [10, 20, 30],
                  [5, 20, 8],
                  [20, 8, 5],
                  [10, 20, 8]])
result = np.array([])
for a in range(0, len(act_func)):
  for n in range(0, nodes.shape[0]):

    #set up a sequential neural network
    model = Sequential()
    model.add(Dense(units=nodes[n][0], activation=act_func[a], input_dim=6)) 
    model.add(Dense(units=nodes[n][1], activation=act_func[a]))
    model.add(Dense(units=nodes[n][2], activation=act_func[a]))
    model.add(Dense(units=2, activation='sigmoid')) 
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.fit(X_train2, yt_binary2, epochs=250, batch_size=32)
    y_pred2 = model.predict(X_test2)
    y_pred2 = np.argmax(y_pred2, axis=-1)

    nTot = len(y_pred2) 
    nMatch = 0 
    for i in range(len(y_pred2)):
      if y_test2[i] == y_pred2[i]:
        nMatch += 1
    #print("With Validation data -- Accuracy: ", 100 * nMatch / nTot, "%")
    result = np.append(result, nMatch / nTot)

result = result.reshape((len(act_func), nodes.shape[0]))
print(result)




Epoch 1/250
84/84 [==============================] - 1s 2ms/step - loss: 0.6895
Epoch 2/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6787
Epoch 3/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6737
Epoch 4/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6709
Epoch 5/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6681
Epoch 6/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6650
Epoch 7/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6621
Epoch 8/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6594
Epoch 9/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6559
Epoch 10/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6538
Epoch 11/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6510
Epoch 12/250
84/84 [==============================] - 0s 2ms/step - loss: 0.6497
Epoch 13/250
84/84 [=================

In [92]:
# 1st iteration - sigmoid, binary_cross entropy
# act_func = ["tanh", "sigmoid", "relu"]
# nodes = np.array([[5, 10],[10, 20], [5, 20], [20,10],[10,30]])

sig_bin = np.array([[0.60757576,0.65378788,0.66136364,0.73030303,0.68560606],
                    [0.60833333,0.61212121,0.60984848,0.60833333,0.61287879],
                    [0.69166667,0.73863636,0.69621212,0.74772727,0.62348485]])
for i in range(0,len(act_func)):
  print(np.average(sig_bin[i]))

# 2nd iteration - softmax, categorical cross entropy
# act_func = ["tanh", "sigmoid", "relu"]
# nodes = np.array([[5, 10],[10, 20], [5, 20], [20,10],[10,30]])

soft_cat = np.array([[0.59242424,0.66287879,0.65681818,0.74924242,0.72272727],
                     [0.61212121,0.61742424,0.60757576,0.62045455,0.61666667],
                     [0.66590909,0.71666667,0.62954545,0.63863636,0.63712121]])

for i in range(0,len(act_func)):
  print(np.average(soft_cat[i]))

0.667727274
0.610303028
0.6995454540000001
0.6768181799999999
0.614848486
0.6575757560000001
